In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon

from mlscorecheck import auc

In [20]:
data = pd.read_csv('raw-single3.csv')

In [21]:
data[:10]

,dataset,acc,sens,spec,auc,best_acc,best_sens,best_spec,threshold,best_threshold,p,n,n_nodes,fprs,tprs,thresholds,classifier,classifier_params
0,bupa,0.695652,0.650000,0.758621,0.753017,0.710145,0.875000,0.482759,0.579710,0.571429,40,29,8,"[0.0, 0.0, 0.10344827586206896, 0.241379310344...","[0.0, 0.025, 0.325, 0.65, 0.875, 0.95, 1.0, 1.0]","[inf, 1.0, 0.8571428571428571, 0.7142857142857...",KNeighborsClassifier,{'n_neighbors': 7}
1,dermatology-6,0.944444,0.941176,1.000000,1.000000,1.000000,1.000000,1.000000,0.944056,0.906440,68,4,3,"[0.0, 0.0, 1.0]","[0.0, 1.0, 1.0]","[inf, 0.9064396244236703, 0.16932266702168314]",RandomForestClassifier,"{'max_depth': 3, 'random_state': 5}"
2,glass0,0.720930,0.655172,0.857143,0.834975,0.813953,0.862069,0.714286,0.672515,0.500000,29,14,8,"[0.0, 0.0, 0.14285714285714285, 0.285714285714...","[0.0, 0.5517241379310345, 0.6551724137931034, ...","[inf, 0.875, 0.75, 0.625, 0.5, 0.375, 0.25, 0.0]",KNeighborsClassifier,{'n_neighbors': 8}
3,yeast1,0.754209,0.810427,0.616279,0.805081,0.781145,0.895735,0.500000,0.711036,0.618373,211,86,95,"[0.0, 0.0, 0.023255813953488372, 0.02325581395...","[0.0, 0.13744075829383887, 0.13744075829383887...","[inf, 0.843755544280719, 0.8432948743550586, 0...",SVC,"{'probability': True, 'C': 0.24431899842918103}"
4,iris0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.333333,1.000000,10,20,3,"[0.0, 0.0, 1.0]","[0.0, 1.0, 1.0]","[inf, 1.0, 0.0]",DecisionTreeClassifier,"{'max_depth': 7, 'random_state': 5}"
5,ecoli1,0.926471,0.923077,0.937500,0.969351,0.926471,0.923077,0.937500,0.772388,0.833333,52,16,8,"[0.0, 0.0, 0.0625, 0.25, 0.5, 0.625, 0.8125, 1.0]","[0.0, 0.8076923076923077, 0.9230769230769231, ...","[inf, 1.0, 0.8333333333333334, 0.6666666666666...",KNeighborsClassifier,{'n_neighbors': 6}
6,led7digit-0-2-4-6-7-8-9_vs_1,0.921348,0.571429,0.951220,0.766551,0.943820,0.571429,0.975610,0.084746,0.727273,7,82,5,"[0.0, 0.012195121951219513, 0.0243902439024390...","[0.0, 0.14285714285714285, 0.5714285714285714,...","[inf, 1.0, 0.7272727272727273, 0.3333333333333...",DecisionTreeClassifier,"{'max_depth': 9, 'random_state': 5}"
7,saheart,0.666667,0.672131,0.656250,0.727459,0.752688,0.852459,0.562500,0.653117,0.550000,61,32,8,"[0.0, 0.03125, 0.0625, 0.125, 0.34375, 0.4375,...","[0.0, 0.04918032786885246, 0.3442622950819672,...","[inf, 1.0, 0.9651162790697675, 0.9523809523809...",DecisionTreeClassifier,"{'max_depth': 4, 'random_state': 5}"
8,wdbc,0.947368,0.880952,0.986111,0.995040,0.964912,0.928571,0.986111,0.373626,0.361044,42,72,9,"[0.0, 0.0, 0.013888888888888888, 0.01388888888...","[0.0, 0.8095238095238095, 0.8095238095238095, ...","[inf, 0.9053006913925485, 0.45639162765411245,...",SVC,"{'probability': True, 'C': 0.29091890509477253}"
9,appendicitis,0.727273,0.722222,0.750000,0.840278,0.954545,1.000000,0.750000,0.797619,0.500000,18,4,4,"[0.0, 0.25, 0.25, 1.0]","[0.0, 0.7222222222222222, 1.0, 1.0]","[inf, 1.0, 0.5, 0.0]",KNeighborsClassifier,{'n_neighbors': 2}


In [22]:
data.columns

Index(['dataset', 'acc', 'sens', 'spec', 'auc', 'best_acc', 'best_sens',
       'best_spec', 'threshold', 'best_threshold', 'p', 'n', 'n_nodes', 'fprs',
       'tprs', 'thresholds', 'classifier', 'classifier_params'],
      dtype='object')

In [23]:
lower_bounds = ['min', 'rmin', 'grmin', 'amin', 'armin', 'onmin']
#lower_bounds = ['min']
upper_bounds = ['max', 'amax', 'maxa']
#upper_bounds = ['max']

In [24]:
def wrapper(func, **kwargs):
    try:
        return func(**kwargs)[0]
    except Exception as exc:
        return str(exc)

In [25]:
for lb in lower_bounds:
    data[f'auc_{lb}'] = data.apply(
        lambda row:
        wrapper(auc.auc_lower_from,
        scores={
            'acc': row['acc'],
            'sens': row['sens'],
            'spec': row['spec']
        },
        p=row['p'],
        n=row['n'],
        eps=1e-4,
        lower=lb),
        axis=1
    )

    data[f'auc_{lb}_best'] = data.apply(
        lambda row:
        wrapper(auc.auc_lower_from,
        scores={
            'acc': row['best_acc'],
            'sens': row['best_sens'],
            'spec': row['best_spec']
        },
        p=row['p'],
        n=row['n'],
        eps=1e-4,
        lower=lb),
        axis=1
    )

In [26]:
for ub in upper_bounds:
    data[f'auc_{ub}'] = data.apply(
        lambda row:
        wrapper(
        auc.auc_upper_from,
        scores={
            'acc': row['acc'] if ub != 'maxa' else row['best_acc'],
            'sens': row['sens'] if ub != 'maxa' else row['best_sens'],
            'spec': row['spec'] if ub != 'maxa' else row['best_spec']
        },
        p=row['p'],
        n=row['n'],
        eps=1e-4,
        upper=ub),
        axis=1
    )

    data[f'auc_{ub}_best'] = data.apply(
        lambda row:
        wrapper(
        auc.auc_upper_from,
        scores={
            'acc': row['best_acc'],
            'sens': row['best_sens'],
            'spec': row['best_spec']
        },
        p=row['p'],
        n=row['n'],
        eps=1e-4,
        upper=ub),
        axis=1
    )

In [27]:
lower_bounds = ['min', 'rmin']
upper_bounds = ['max', 'rmax', 'onmax']

In [28]:
for lb in lower_bounds:
    data[f'acc_{lb}'] = data.apply(
        lambda row:
        wrapper(auc.acc_lower_from,
        scores={
            'acc': row['acc'],
            'sens': row['sens'],
            'spec': row['spec'],
            'auc': row['auc']
        },
        p=row['p'],
        n=row['n'],
        eps=1e-4,
        lower=lb),
        axis=1
    )

In [29]:
for ub in upper_bounds:
    data[f'acc_{ub}'] = data.apply(
        lambda row:
        wrapper(auc.acc_upper_from,
        scores={
            'acc': row['acc'],
            'sens': row['sens'],
            'spec': row['spec'],
            'auc': row['auc']
        },
        p=row['p'],
        n=row['n'],
        eps=1e-4,
        upper=ub),
        axis=1
    )

In [30]:
lower_bounds = ['min']
upper_bounds = ['max', 'rmax', 'onmax']

In [31]:
for lb in lower_bounds:
    data[f'max_acc_{lb}'] = data.apply(
        lambda row:
        wrapper(auc.max_acc_lower_from,
        scores={
            'acc': row['acc'],
            'sens': row['sens'],
            'spec': row['spec'],
            'auc': row['auc']
        },
        p=row['p'],
        n=row['n'],
        eps=1e-4,
        lower=lb),
        axis=1
    )

In [32]:
for ub in upper_bounds:
    data[f'max_acc_{ub}'] = data.apply(
        lambda row:
        wrapper(auc.max_acc_upper_from,
        scores={
            'acc': row['acc'],
            'sens': row['sens'],
            'spec': row['spec'],
            'auc': row['auc']
        },
        p=row['p'],
        n=row['n'],
        eps=1e-4,
        upper=ub),
        axis=1
    )

In [33]:
data.columns

Index(['dataset', 'acc', 'sens', 'spec', 'auc', 'best_acc', 'best_sens',
       'best_spec', 'threshold', 'best_threshold', 'p', 'n', 'n_nodes', 'fprs',
       'tprs', 'thresholds', 'classifier', 'classifier_params', 'auc_min',
       'auc_min_best', 'auc_rmin', 'auc_rmin_best', 'auc_grmin',
       'auc_grmin_best', 'auc_amin', 'auc_amin_best', 'auc_armin',
       'auc_armin_best', 'auc_onmin', 'auc_onmin_best', 'auc_max',
       'auc_max_best', 'auc_amax', 'auc_amax_best', 'auc_maxa',
       'auc_maxa_best', 'acc_min', 'acc_rmin', 'acc_max', 'acc_rmax',
       'acc_onmax', 'max_acc_min', 'max_acc_max', 'max_acc_rmax',
       'max_acc_onmax'],
      dtype='object')

In [34]:
data.head()

,dataset,acc,sens,spec,auc,best_acc,best_sens,best_spec,threshold,best_threshold,...,auc_maxa_best,acc_min,acc_rmin,acc_max,acc_rmax,acc_onmax,max_acc_min,max_acc_max,max_acc_rmax,max_acc_onmax
0,bupa,0.695652,0.650000,0.758621,0.753017,0.710145,0.875000,0.482759,0.579710,0.571429,...,0.827705,'float' object is not subscriptable,'float' object is not subscriptable,'float' object is not subscriptable,invalid index to scalar variable.,'float' object is not subscriptable,0.653011,0.896238,0.878747,0.792475
1,dermatology-6,0.944444,0.941176,1.000000,1.000000,1.000000,1.000000,1.000000,0.944056,0.906440,...,1.000000,'float' object is not subscriptable,'float' object is not subscriptable,'float' object is not subscriptable,invalid index to scalar variable.,'float' object is not subscriptable,0.996761,1.000000,1.000000,1.000000
2,glass0,0.720930,0.655172,0.857143,0.834975,0.813953,0.862069,0.714286,0.672515,0.500000,...,0.921267,'float' object is not subscriptable,'float' object is not subscriptable,'float' object is not subscriptable,invalid index to scalar variable.,'float' object is not subscriptable,0.730713,0.946304,0.940948,0.892607
3,yeast1,0.754209,0.810427,0.616279,0.805081,0.781145,0.895735,0.500000,0.711036,0.618373,...,0.883690,'float' object is not subscriptable,'float' object is not subscriptable,'float' object is not subscriptable,invalid index to scalar variable.,'float' object is not subscriptable,0.716739,0.943588,0.936660,0.887176
4,iris0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.333333,1.000000,...,1.000000,'float' object is not subscriptable,'float' object is not subscriptable,'float' object is not subscriptable,invalid index to scalar variable.,'float' object is not subscriptable,0.993333,1.000000,1.000000,1.000000


In [35]:
data.to_csv('processed-single3.csv', index=False)

In [36]:
data.iloc[0]

dataset                                                           bupa
acc                                                           0.695652
sens                                                              0.65
spec                                                          0.758621
auc                                                           0.753017
best_acc                                                      0.710145
best_sens                                                        0.875
best_spec                                                     0.482759
threshold                                                      0.57971
best_threshold                                                0.571429
p                                                                   40
n                                                                   29
n_nodes                                                              8
fprs                 [0.0, 0.0, 0.10344827586206896, 0.241379310344...
tprs  